In [1]:
from skimage import transform, filters, exposure
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from sklearn.cross_validation import train_test_split
from keras.layers.convolutional import Conv2D

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import os
import glob
import pandas as pd
import math
from PIL import Image
import numpy as np
from scipy.misc import imread, imsave, imresize
from natsort import natsorted
from scipy.misc import imread
import random
random.seed( 42 ) 

# Path of data files
path = "/home/ivan/python/julia/data"
 
# Input image dimensions
img_rows, img_cols = 32, 32
 
 

### Images preprocessing ###

for setType in ["test","train"]:
    # We have to make sure files are sorted according to labels, even if they don't have trailing zeros
    files = natsorted(glob.glob(path + "/"+setType+"/*"))
    """
    if setType == 'test':
        X_test = np.array([np.array(Image.open(fname)) for fname in files])
    else:
        X_train = np.array([np.array(Image.open(fname)) for fname in files])
    """
    if setType == 'test':
        X_test = np.array([np.array(imread(fname, flatten=True)) for fname in files])
    else:
        X_train = np.array([np.array(imread(fname, flatten=True)) for fname in files])
X_test=X_test[:,:,:,np.newaxis]
X_train=X_train[:,:,:,np.newaxis]
X_train /= 255
X_test /= 255
X_test.shape
    

(6220, 20, 20, 1)

In [3]:
def label2int(ch):
    asciiVal = ord(ch)
    if(asciiVal<=57): #0-9
        asciiVal-=48
    elif(asciiVal<=90): #A-Z
        asciiVal-=55
    else: #a-z
        asciiVal-=61
    return asciiVal
    
def int2label(i):
    if(i<=9): #0-9
        i+=48
    elif(i<=35): #A-Z
        i+=55
    else: #a-z
        i+=61
    return chr(i)
# Load labels
y_train = pd.read_csv("/home/ivan/python/julia/trainLabels.csv").values[:,1] #Keep only label
 
# Convert labels to one-hot vectors
Y_train = np.zeros((y_train.shape[0], len(np.unique(y_train))))
 
for i in range(y_train.shape[0]):
    Y_train[i][label2int(y_train[i])] = 1 # One-hot
Y_train_all = Y_train.copy()
X_train_all = X_train.copy()

In [4]:
X_train, X_val, Y_train, Y_val = \
        train_test_split(X_train_all, Y_train_all, test_size=0.2, stratify=np.argmax(Y_train_all, axis=1))

In [5]:
nb_classes = 62 # A-Z, a-z and 0-9
nb_epoch = 500
img_rows, img_cols = 20, 20
model = Sequential()

#model.add(Convolution2D(3, 3, 128,  border_mode='same', init='he_normal', activation = 'relu', input_shape=(1, img_rows, img_cols)))
model.add(Conv2D(32, (3, 3), kernel_initializer="he_normal", activation="relu", input_shape=(img_rows, img_cols, 1), padding="same"))
model.add(Conv2D(32, (3, 3), kernel_initializer="he_normal", activation="relu", padding="same"))

model.add(MaxPooling2D(data_format="channels_last", pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), kernel_initializer="he_normal", activation="relu", padding="same"))
model.add(Conv2D(64, (3, 3), kernel_initializer="he_normal", activation="relu", padding="same"))

model.add(MaxPooling2D(data_format="channels_last", pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), kernel_initializer="he_normal", activation="relu", padding="same"))
model.add(Conv2D(128, (3, 3), kernel_initializer="he_normal", activation="relu", padding="same"))
model.add(Conv2D(128, (3, 3), kernel_initializer="he_normal", activation="relu", padding="same"))

model.add(MaxPooling2D(data_format="channels_last", pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(2048, kernel_initializer="he_normal", activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(2048, kernel_initializer="he_normal", activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, kernel_initializer="he_normal", activation="softmax"))

In [6]:
### LEARNING ###
 
# First, use AdaDelta for some epochs because AdaMax gets stuck
model.compile(loss='categorical_crossentropy', 
              optimizer='adadelta',  
              metrics=["accuracy"])

In [7]:
# 20 epochs is sufficient
model.fit(X_train, Y_train, batch_size=128,
                    #nb_epoch=20, 
                    epochs=20,
                    validation_data=(X_val, Y_val),
                    verbose=1)

Train on 5026 samples, validate on 1257 samples
Epoch 1/20
5026/5026 [==============================] - 37s - loss: 3.8828 - acc: 0.0611 - val_loss: 3.8377 - val_acc: 0.0660
Epoch 2/20
5026/5026 [==============================] - 37s - loss: 3.7641 - acc: 0.0965 - val_loss: 3.6786 - val_acc: 0.1233
Epoch 3/20
5026/5026 [==============================] - 37s - loss: 3.5462 - acc: 0.1590 - val_loss: 3.2171 - val_acc: 0.2299
Epoch 4/20
5026/5026 [==============================] - 37s - loss: 2.9507 - acc: 0.2969 - val_loss: 3.0054 - val_acc: 0.2792
Epoch 5/20
5026/5026 [==============================] - 39s - loss: 2.3524 - acc: 0.3991 - val_loss: 2.2795 - val_acc: 0.3930
Epoch 6/20
5026/5026 [==============================] - 38s - loss: 1.9426 - acc: 0.4895 - val_loss: 1.9688 - val_acc: 0.5004
Epoch 7/20
5026/5026 [==============================] - 37s - loss: 1.6416 - acc: 0.5575 - val_loss: 1.8071 - val_acc: 0.5076
Epoch 8/20
5026/5026 [==============================] - 38s - loss: 1.

In [10]:
# Parametrize the image augmentation class
datagen = ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.15,
    height_shift_range = 0.15,
    shear_range = 0.4,
    zoom_range = 0.3,                    
    channel_shift_range = 0.1)

In [11]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adamax',  
              metrics=["accuracy"])

# We want to keep the best model. This callback will store 
# in a file the weights of the model with the highest validation accuracy  
saveBestModel = ModelCheckpoint("best.kerasModelWeights", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True)

# Make the model learn using the image generator
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=128),
                samples_per_epoch=len(X_train),
                epochs=500, 
                validation_data=(X_val, Y_val),
                callbacks=[saveBestModel],
                verbose=1)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., verbose=1, steps_per_epoch=39, epochs=500, callbacks=[<keras.ca..., validation_data=(array([[[...)`
  from ipykernel import kernelapp as app


Epoch 1/500
39/39 [==============================] - 35s - loss: 2.8159 - acc: 0.3081 - val_loss: 1.3249 - val_acc: 0.6396
Epoch 2/500
39/39 [==============================] - 34s - loss: 2.2837 - acc: 0.4060 - val_loss: 1.1810 - val_acc: 0.6683
Epoch 3/500
39/39 [==============================] - 37s - loss: 2.0616 - acc: 0.4506 - val_loss: 1.0694 - val_acc: 0.7001
Epoch 4/500
39/39 [==============================] - 36s - loss: 1.9268 - acc: 0.4836 - val_loss: 1.0077 - val_acc: 0.7112
Epoch 5/500
39/39 [==============================] - 35s - loss: 1.9108 - acc: 0.4923 - val_loss: 0.9602 - val_acc: 0.7414
Epoch 6/500
39/39 [==============================] - 34s - loss: 1.8133 - acc: 0.5063 - val_loss: 0.9435 - val_acc: 0.7295
Epoch 7/500
39/39 [==============================] - 35s - loss: 1.7740 - acc: 0.5212 - val_loss: 0.9306 - val_acc: 0.7399
Epoch 8/500
39/39 [==============================] - 35s - loss: 1.6529 - acc: 0.5568 - val_loss: 0.9364 - val_acc: 0.7399
Epoch 9/500
39/3

39/39 [==============================] - 35s - loss: 1.1505 - acc: 0.6692 - val_loss: 0.7413 - val_acc: 0.7852
Epoch 33/500
39/39 [==============================] - 35s - loss: 1.1319 - acc: 0.6691 - val_loss: 0.7193 - val_acc: 0.7892
Epoch 34/500
39/39 [==============================] - 35s - loss: 1.0987 - acc: 0.6802 - val_loss: 0.7441 - val_acc: 0.7947
Epoch 35/500
39/39 [==============================] - 35s - loss: 1.0962 - acc: 0.6828 - val_loss: 0.7547 - val_acc: 0.7669
Epoch 36/500
39/39 [==============================] - 35s - loss: 1.0858 - acc: 0.6815 - val_loss: 0.7319 - val_acc: 0.7844
Epoch 37/500
39/39 [==============================] - 35s - loss: 1.0631 - acc: 0.6893 - val_loss: 0.7440 - val_acc: 0.7836
Epoch 38/500
39/39 [==============================] - 35s - loss: 1.0576 - acc: 0.6882 - val_loss: 0.7780 - val_acc: 0.7693
Epoch 39/500
39/39 [==============================] - 35s - loss: 1.0489 - acc: 0.6880 - val_loss: 0.7235 - val_acc: 0.7892
Epoch 40/500
39/39 [=

39/39 [==============================] - 72s - loss: 0.8340 - acc: 0.7366 - val_loss: 0.6873 - val_acc: 0.8003
Epoch 67/500
39/39 [==============================] - 73s - loss: 0.8493 - acc: 0.7378 - val_loss: 0.6968 - val_acc: 0.7916
Epoch 68/500
39/39 [==============================] - 73s - loss: 0.8338 - acc: 0.7430 - val_loss: 0.6828 - val_acc: 0.8107
Epoch 69/500
39/39 [==============================] - 76s - loss: 0.8244 - acc: 0.7423 - val_loss: 0.7209 - val_acc: 0.7900
Epoch 70/500
39/39 [==============================] - 74s - loss: 0.8237 - acc: 0.7470 - val_loss: 0.6983 - val_acc: 0.8011
Epoch 71/500
39/39 [==============================] - 71s - loss: 0.8294 - acc: 0.7459 - val_loss: 0.7114 - val_acc: 0.7892
Epoch 72/500
39/39 [==============================] - 76s - loss: 0.8041 - acc: 0.7523 - val_loss: 0.7143 - val_acc: 0.7884
Epoch 73/500
39/39 [==============================] - 72s - loss: 0.8376 - acc: 0.7344 - val_loss: 0.7336 - val_acc: 0.7940
Epoch 74/500
39/39 [=

39/39 [==============================] - 67s - loss: 0.6803 - acc: 0.7719 - val_loss: 0.6694 - val_acc: 0.8051
Epoch 101/500
39/39 [==============================] - 67s - loss: 0.6995 - acc: 0.7734 - val_loss: 0.7029 - val_acc: 0.8019
Epoch 102/500
39/39 [==============================] - 67s - loss: 0.6731 - acc: 0.7840 - val_loss: 0.6889 - val_acc: 0.8051
Epoch 103/500
39/39 [==============================] - 68s - loss: 0.6626 - acc: 0.7852 - val_loss: 0.6614 - val_acc: 0.8075
Epoch 104/500
39/39 [==============================] - 68s - loss: 0.6727 - acc: 0.7850 - val_loss: 0.6996 - val_acc: 0.8091
Epoch 105/500
39/39 [==============================] - 67s - loss: 0.6653 - acc: 0.7798 - val_loss: 0.6822 - val_acc: 0.7971
Epoch 106/500
39/39 [==============================] - 68s - loss: 0.6616 - acc: 0.7839 - val_loss: 0.6962 - val_acc: 0.7947
Epoch 107/500
39/39 [==============================] - 68s - loss: 0.6628 - acc: 0.7846 - val_loss: 0.7216 - val_acc: 0.7820
Epoch 108/500


...

39/39 [==============================] - 34s - loss: 0.3552 - acc: 0.8801 - val_loss: 0.7283 - val_acc: 0.8067
Epoch 338/500
39/39 [==============================] - 34s - loss: 0.3501 - acc: 0.8825 - val_loss: 0.7804 - val_acc: 0.8019
Epoch 339/500
39/39 [==============================] - 34s - loss: 0.3497 - acc: 0.8859 - val_loss: 0.7614 - val_acc: 0.8059
Epoch 340/500
39/39 [==============================] - 34s - loss: 0.3338 - acc: 0.8858 - val_loss: 0.7413 - val_acc: 0.8059
Epoch 341/500
39/39 [==============================] - 34s - loss: 0.3537 - acc: 0.8787 - val_loss: 0.7864 - val_acc: 0.8027
Epoch 342/500
39/39 [==============================] - 34s - loss: 0.3347 - acc: 0.8866 - val_loss: 0.7632 - val_acc: 0.8059
Epoch 343/500
39/39 [==============================] - 35s - loss: 0.3189 - acc: 0.8895 - val_loss: 0.7643 - val_acc: 0.8123
Epoch 344/500
39/39 [==============================] - 34s - loss: 0.3403 - acc: 0.8863 - val_loss: 0.7987 - val_acc: 0.8091
Epoch 345/500


39/39 [==============================] - 35s - loss: 0.3128 - acc: 0.8936 - val_loss: 0.7490 - val_acc: 0.8170
Epoch 372/500
39/39 [==============================] - 34s - loss: 0.3281 - acc: 0.8861 - val_loss: 0.7740 - val_acc: 0.7995
Epoch 373/500
39/39 [==============================] - 34s - loss: 0.2990 - acc: 0.9009 - val_loss: 0.7522 - val_acc: 0.8162
Epoch 374/500
39/39 [==============================] - 34s - loss: 0.3160 - acc: 0.8945 - val_loss: 0.7408 - val_acc: 0.8274
Epoch 375/500
39/39 [==============================] - 34s - loss: 0.3282 - acc: 0.8895 - val_loss: 0.7526 - val_acc: 0.8091
Epoch 376/500
39/39 [==============================] - 34s - loss: 0.3112 - acc: 0.8906 - val_loss: 0.7514 - val_acc: 0.8123
Epoch 377/500
39/39 [==============================] - 35s - loss: 0.3151 - acc: 0.8915 - val_loss: 0.7677 - val_acc: 0.8075
Epoch 378/500
39/39 [==============================] - 34s - loss: 0.3210 - acc: 0.8913 - val_loss: 0.7542 - val_acc: 0.8059
Epoch 379/500


39/39 [==============================] - 34s - loss: 0.3086 - acc: 0.8986 - val_loss: 0.7795 - val_acc: 0.8011
Epoch 406/500
39/39 [==============================] - 34s - loss: 0.3138 - acc: 0.8911 - val_loss: 0.7559 - val_acc: 0.8019
Epoch 407/500
39/39 [==============================] - 34s - loss: 0.3191 - acc: 0.8869 - val_loss: 0.7829 - val_acc: 0.8075
Epoch 408/500
39/39 [==============================] - 34s - loss: 0.2908 - acc: 0.9016 - val_loss: 0.7793 - val_acc: 0.8154
Epoch 409/500
39/39 [==============================] - 34s - loss: 0.3113 - acc: 0.8944 - val_loss: 0.7711 - val_acc: 0.8035
Epoch 410/500
39/39 [==============================] - 34s - loss: 0.3071 - acc: 0.8900 - val_loss: 0.8017 - val_acc: 0.8027
Epoch 411/500
39/39 [==============================] - 35s - loss: 0.2964 - acc: 0.8972 - val_loss: 0.8271 - val_acc: 0.8107
Epoch 412/500
39/39 [==============================] - 34s - loss: 0.3202 - acc: 0.8906 - val_loss: 0.8027 - val_acc: 0.8115
Epoch 413/500


39/39 [==============================] - 34s - loss: 0.2894 - acc: 0.8972 - val_loss: 0.8095 - val_acc: 0.8011
Epoch 440/500
39/39 [==============================] - 35s - loss: 0.3127 - acc: 0.8940 - val_loss: 0.7918 - val_acc: 0.8067
Epoch 441/500
39/39 [==============================] - 35s - loss: 0.2994 - acc: 0.8988 - val_loss: 0.7912 - val_acc: 0.8107
Epoch 442/500
39/39 [==============================] - 34s - loss: 0.2850 - acc: 0.9028 - val_loss: 0.7710 - val_acc: 0.7971
Epoch 443/500
39/39 [==============================] - 34s - loss: 0.2763 - acc: 0.9010 - val_loss: 0.7765 - val_acc: 0.8130
Epoch 444/500
39/39 [==============================] - 34s - loss: 0.3109 - acc: 0.8972 - val_loss: 0.8022 - val_acc: 0.8123
Epoch 445/500
39/39 [==============================] - 35s - loss: 0.2795 - acc: 0.8986 - val_loss: 0.7970 - val_acc: 0.8146
Epoch 446/500
39/39 [==============================] - 34s - loss: 0.2697 - acc: 0.9079 - val_loss: 0.7731 - val_acc: 0.8043
Epoch 447/500


39/39 [==============================] - 34s - loss: 0.2712 - acc: 0.9032 - val_loss: 0.8659 - val_acc: 0.8099
Epoch 474/500
39/39 [==============================] - 34s - loss: 0.2720 - acc: 0.9098 - val_loss: 0.8378 - val_acc: 0.8059
Epoch 475/500
39/39 [==============================] - 34s - loss: 0.2729 - acc: 0.9094 - val_loss: 0.8409 - val_acc: 0.8115
Epoch 476/500
39/39 [==============================] - 34s - loss: 0.2823 - acc: 0.9025 - val_loss: 0.8174 - val_acc: 0.8115
Epoch 477/500
39/39 [==============================] - 35s - loss: 0.2870 - acc: 0.9013 - val_loss: 0.7619 - val_acc: 0.8099
Epoch 478/500
39/39 [==============================] - 34s - loss: 0.2840 - acc: 0.9065 - val_loss: 0.7872 - val_acc: 0.8107
Epoch 479/500
39/39 [==============================] - 34s - loss: 0.2614 - acc: 0.9145 - val_loss: 0.7957 - val_acc: 0.8138
Epoch 480/500
39/39 [==============================] - 35s - loss: 0.2810 - acc: 0.9030 - val_loss: 0.8182 - val_acc: 0.8083
Epoch 481/500


In [12]:
# Predict the class (give the index in the one-hot vector of the most probable class)
Y_test_pred = model.predict_classes(X_test)

# Translate integers to character labels
vInt2label = np.vectorize(int2label)
Y_test_pred = vInt2label(Y_test_pred)

# Save the predicitions in Kaggle format
np.savetxt(path+"/CNN_pred1.csv", np.c_[range(6284,len(Y_test_pred)+6284),Y_test_pred], delimiter=',', header = 'ID,Class', comments = '', fmt='%s')



6208/6220 [============================>.] - ETA: 0s